FEELINGS

In [ ]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Baixar o lexicon do VADER
#nltk.download('vader_lexicon')

# Caminhos para os arquivos Excel
xgsfacebook_path = r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\XGS Facebook.xlsx'
sopfacebook_path = r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\SoP Facebook.xlsx'

# Carregar os arquivos Excel em DataFrames
xgsfacebook_df = pd.read_excel(xgsfacebook_path)
sopfacebook_df = pd.read_excel(sopfacebook_path)

# Inicializar o analisador de sentimentos do VADER
sia = SentimentIntensityAnalyzer()

# Função para calcular o sentimento e aplicar rótulos
def get_sentiment(text):
    score = sia.polarity_scores(text)
    if score['compound'] > 0.05:
        return 'Positive'
    elif score['compound'] < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Garantir que os comentários são strings e lidar com valores ausentes
xgsfacebook_df['Comments'] = xgsfacebook_df['Comments'].astype(str).fillna('')
sopfacebook_df['Comments'] = sopfacebook_df['Comments'].astype(str).fillna('')

# Aplicar a função aos comentários nos DataFrames
xgsfacebook_df['Sentiment'] = xgsfacebook_df['Comments'].apply(get_sentiment)
sopfacebook_df['Sentiment'] = sopfacebook_df['Comments'].apply(get_sentiment)

# Exibir os DataFrames com a nova coluna de sentimento
print(xgsfacebook_df.head())
print(sopfacebook_df.head())

# Salvar os DataFrames de volta nos arquivos Excel
xgsfacebook_df.to_excel(r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\Feelings\XGS Facebook Feelings.xlsx', index=False)
sopfacebook_df.to_excel(r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\Feelings\SoP Facebook Feelings.xlsx', index=False)


EMOTIONS

In [ ]:
import pandas as pd
import text2emotion as te
from tqdm import tqdm

# Caminhos para os arquivos Excel
xgsfacebook_path = r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\Feelings\XGS Facebook Feelings.xlsx'
sopfacebook_path = r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\Feelings\SoP Facebook Feelings.xlsx'

# Carregar os arquivos Excel em DataFrames
xgsfacebook_df = pd.read_excel(xgsfacebook_path)
sopfacebook_df = pd.read_excel(sopfacebook_path)

# Função para calcular a emoção dominante
def get_dominant_emotion(text):
    try:
        emotions = te.get_emotion(text)
        # Retornar a emoção com o maior valor
        dominant_emotion = max(emotions, key=emotions.get)
        return dominant_emotion
    except Exception as e:
        print(f"Error processing text: {text}")
        print(f"Exception: {e}")
        return None

# Usar tqdm para acompanhar o progresso
tqdm.pandas()

# Aplicar a função aos comentários nos DataFrames com progresso
xgsfacebook_df['Emotion'] = xgsfacebook_df['Comments'].progress_apply(get_dominant_emotion)
sopfacebook_df['Emotion'] = sopfacebook_df['Comments'].progress_apply(get_dominant_emotion)

# Exibir os DataFrames com a nova coluna de sentimento dominante
print(xgsfacebook_df.head())
print(sopfacebook_df.head())

# Se desejar, salvar os DataFrames de volta nos arquivos Excel
xgsfacebook_df.to_excel(r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\Feelings\XGS Facebook Emotions.xlsx', index=False)
sopfacebook_df.to_excel(r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\Feelings\SoP Facebook Emotions.xlsx', index=False)


KEYWORD

In [ ]:
import pandas as pd
from collections import Counter
from tqdm import tqdm
import re

# Caminhos para os arquivos Excel
xgsfacebook_path = r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Definitive Edition\XGS Facebook Keyword.xlsx'
sopfacebook_path = r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Definitive Edition\SoP Facebook Keyword.xlsx'

# Carregar os arquivos Excel em DataFrames
xgsfacebook_df = pd.read_excel(xgsfacebook_path)
sopfacebook_df = pd.read_excel(sopfacebook_path)

# Definir uma lista de palavras-chave relevantes
relevant_keywords = [
    'Halo', 'Forza', 'Elder Scrolls', 'Starfield', 'Fable', 'Xbox', 'PlayStation', 
    'Sony', 'Microsoft', 'State of Play', 'Xbox Games Showcase', 'Game', 'Gamespass', 'Game Pass', 'Release',
    'Announcement', 'Exclusive', 'Console', 'Cod', 'ConnectionAbortedError',
    'Call of Duty', 'BO', 'BO6', 'Doom', 'State of Decay 3', 
    'Dragon Age', 'Dragon Age The Veilguard', 'Starfield Shattered Space', 'Fallout 76 Skyline Valley', 
    'Expedition 33', 'Expedition', 'South of Midnight', 'World of Warcraft', 
    'Metal Gear Solid', 'Sea of Thieves', 'Flintlock', 
    'Age of Mythology', 'Perfect Dark', 'Diablo', 'FragPunk', 
    'Winter Burrow', 'Mixtape', 'Microsoft Flight Simulator', 'Flight Simulator'
    'The Elder Scrolls Online', 'Elder Scrolls', 'Life is Strange', 
    'Indiana Jones', 'Mecha Break', 'Wuchang', 
    'Avowed', 'Atomfall', 'Assassin’s Creed', 'AC', 'S.T.A.L.K.E.R 2', 'Stalker',
    'Gears of War', 'Gears', 'Disappointed', 'Disappointing', 'Showcase', 'Astrobot', 'Astro', 'Concord',
    'GOW', 'GoW', 'God of War', 'Until Dawn', 'Skydance', 'Behemoth', 'PoE', 'PoE2', 'Path of Exile',
    'Monster Hunter', 'Silent Hill', 'Alien', 'Suck', 'Sucked', 'Bored', 'Boring', 'Excellent', 'Horrible', 'Mario',

    'halo', 'forza', 'elder scrolls', 'starfield', 'fable', 'xbox', 'playstation', 
    'sony', 'microsoft', 'state of play', 'xbox games showcase', 'game', 'gamespass', 'game pass', 'release',
    'announcement', 'exclusive', 'console', 'cod', 'connectionabortederror',
    'call of duty', 'bo', 'bo6', 'doom', 'state of decay 3', 
    'dragon age', 'dragon age the veilguard', 'starfield shattered space', 'fallout 76 skyline valley', 
    'expedition 33', 'expedition', 'south of midnight', 'world of warcraft', 
    'metal gear solid', 'sea of thieves', 'flintlock', 
    'age of mythology', 'perfect dark', 'diablo', 'fragpunk', 
    'winter burrow', 'mixtape', 'microsoft flight simulator', 'flight simulator'
    'the elder scrolls online', 'elder scrolls', 'life is strange', 
    'indiana jones', 'mecha break', 'wuchang', 
    'avowed', 'atomfall', 'assassin’s creed', 'ac', 's.t.a.l.k.e.r 2', 'stalker',
    'gears of war', 'gears', 'disappointed', 'disappointing', 'showcase', 'astrobot', 'astro', 'concord',
    'gow', 'god of war', 'until dawn', 'skydance', 'behemoth', 'poe', 'poe2', 'path of exile',
    'monster hunter', 'silent hill', 'alien', 'suck', 'sucked', 'bored', 'boring', 'excellent', 'horrible', 'mario',

    'HALO', 'FORZA', 'ELDER SCROLLS', 'STARFIELD', 'FABLE', 'XBOX', 'PLAYSTATION', 
    'SONY', 'MICROSOFT', 'STATE OF PLAY', 'XBOX GAMES SHOWCASE', 'GAME', 'GAMESPASS', 'GAME PASS', 'RELEASE',
    'ANNOUNCEMENT', 'EXCLUSIVE', 'CONSOLE', 'COD', 'CONNECTIONABORTEDERROR',
    'CALL OF DUTY', 'BO', 'BO6', 'DOOM', 'STATE OF DECAY 3', 
    'DRAGON AGE THE VEILGUARD', 'STARFIELD SHATTERED SPACE', 'FALLOUT 76 SKYLINE VALLEY', 
    'EXPEDITION 33', 'EXPEDITION', 'SOUTH OF MIDNIGHT', 'WORLD OF WARCRAFT', 
    'METAL GEAR SOLID', 'SEA OF THIEVES', 'FLINTLOCK', 
    'AGE OF MYTHOLOGY', 'PERFECT DARK', 'DIABLO', 'FRAGPUNK', 
    'WINTER BURROW', 'MIXTAPE', 'MICROSOFT FLIGHT SIMULATOR', 
    'THE ELDER SCROLLS ONLINE', 'LIFE IS STRANGE', 
    'INDIANA JONES', 'MECHA BREAK', 'WUCHANG', 
    'AVOWED', 'ATOMFALL', 'ASSASSIN’S CREED', 'S.T.A.L.K.E.R 2', 'STALKER',
    'GEARS OF WAR', 'GEARS', 'DISAPPOINTED', 'DISAPPOINTING', 'SHOWCASE', 'ASTROBOT', 'ASTRO', 'CONCORD',
    'GOW', 'GOD OF WAR', 'UNTIL DAWN', 'SKYDANCE', 'BEHEMOTH', 'POE', 'POE2', 'PATH OF EXILE',
    'MONSTER HUNTER', 'SILENT HILL', 'ALIEN', 'SUCK', 'SUCKED', 'BORED', 'BORING', 'EXCELLENT', 'HORRIBLE', 'MARIO'
]

# Função para identificar a palavra-chave mais comum em um comentário
def get_most_common_keyword(text, keywords):
    # Remover pontuação e converter para minúsculas
    text = re.sub(r'[^\w\s]', '', text).lower()
    words = text.split()
    # Criar uma lista de contadores para palavras compostas e simples
    keyword_counts = Counter()
    for keyword in keywords:
        if ' ' in keyword:
            if keyword.lower() in text:
                keyword_counts[keyword] += 1
        else:
            keyword_counts.update(word for word in words if word == keyword.lower())
    if keyword_counts:
        most_common_keyword, _ = keyword_counts.most_common(1)[0]
        return most_common_keyword
    return None

# Usar tqdm para acompanhar o progresso
tqdm.pandas()

# Aplicar a função aos comentários nos DataFrames com progresso
xgsfacebook_df['Most Common Keyword'] = xgsfacebook_df['Comments'].progress_apply(lambda x: get_most_common_keyword(x, relevant_keywords))
sopfacebook_df['Most Common Keyword'] = sopfacebook_df['Comments'].progress_apply(lambda x: get_most_common_keyword(x, relevant_keywords))

# Exibir os DataFrames com a nova coluna de palavra-chave mais comum
print(xgsfacebook_df.head())
print(sopfacebook_df.head())

# Se desejar, salvar os DataFrames de volta nos arquivos Excel
xgsfacebook_df.to_excel(r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Definitive Edition\XGS Facebook Keyword.xlsx', index=False)
sopfacebook_df.to_excel(r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Definitive Edition\SoP Facebook Keyword.xlsx', index=False)


SENTIMENT CORRECTION

In [ ]:
import pandas as pd

# Caminhos para os arquivos Excel
xgsfacebook_path = r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\Feelings\XGS Facebook Keyword.xlsx'
sopfacebook_path = r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\Feelings\SoP Facebook Keyword.xlsx'

# Carregar os arquivos Excel em DataFrames
xgsfacebook_df = pd.read_excel(xgsfacebook_path)
sopfacebook_df = pd.read_excel(sopfacebook_path)

# Função de correção do Sentimento
def correct_sentiment(row):
    if row['Emotion'] in ["Sad", "Fear"]:
        return 'Negative'
    elif row['Emotion'] in ["Happy", "Surprise"]:
        return 'Positive'
    return row['Sentiment']

# Aplicar a função aos DataFrames
xgsfacebook_df['Sentiment'] = xgsfacebook_df.apply(correct_sentiment, axis=1)
sopfacebook_df['Sentiment'] = sopfacebook_df.apply(correct_sentiment, axis=1)

#Capitalizar os valores Keyword
xgsfacebook_df['Most Common Keyword'].str.upper()
sopfacebook_df['Most Common Keyword'].str.upper()

# Exibir os DataFrames com a coluna corrigida
print(xgsfacebook_df.head())
print(sopfacebook_df.head())

# Salvar os DataFrames de volta nos arquivos Excel
xgsfacebook_df.to_excel(r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\Feelings\XGS Facebook Keyword.xlsx', index=False)
sopfacebook_df.to_excel(r'C:\Users\Shelton\Documents\VSCode\Python\Clash of Titans\Data\Clean\Main\Feelings\SoP Facebook Keyword.xlsx', index=False)
